In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers,Input,Model,activations,Sequential,callbacks
from keras.utils.vis_utils import plot_model

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
(x_train, y_train_), (x_val, y_val_) = keras.datasets.mnist.load_data()


In [3]:
x_train = np.expand_dims(x_train, -1).astype("float32") / 255
x_val = np.expand_dims(x_val, -1).astype("float32") / 255

In [4]:
def auto_encoder():
    input_shape=(28, 28,1)
    
    input_img = Input(shape=input_shape)
    x      = layers.Conv2D(filters=1,kernel_size=3, padding='same',activation='relu')(input_img)
    x      = layers.BatchNormalization()(x) 
    x      = layers.Activation(activations.relu)(x)
    
    x      = layers.Flatten()(x)
    x      = layers.Dense(150, activation="relu",kernel_initializer='he_normal')(x)
    x      = layers.Dense(30, activation="relu",kernel_initializer='he_normal')(x)
    code   = layers.Dense(9, activation="relu",kernel_initializer='he_normal')(x)

    De_in  = layers.Dense(30, activation="relu",kernel_initializer='he_normal')(code)
    z      = layers.Dense(150, activation="relu",kernel_initializer='he_normal')(De_in)
    z      = layers.Dense(784, activation="relu",kernel_initializer='he_normal')(z)
    
    z      = layers.Reshape((28, 28, 1))(z)

    de_out = layers.Conv2DTranspose(filters=1,kernel_size=1,activation='relu',padding='same')(z)
    
    Encoder=Model(input_img, code)
    Decoder=Model(De_in, de_out)
    Autoencoder=Model(input_img, de_out)

    

    learning_rate=1e-4
    decay_rate = learning_rate / 10
    optimizer   = keras.optimizers.Adam(learning_rate=learning_rate,decay=decay_rate)
    Autoencoder.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_absolute_error'])

    return Encoder,Decoder,Autoencoder


In [5]:
Encoder,Decoder,Autoencoder= auto_encoder()
Autoencoder.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 1)         10        
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 1)        4         
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 28, 28, 1)         0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 150)               117750    
                                                           

2022-05-08 02:27:55.627687: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-08 02:27:55.627745: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (UB): /proc/driver/nvidia/version does not exist
2022-05-08 02:27:55.628412: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
reduce_LR = callbacks.ReduceLROnPlateau(monitor='val_mean_absolute_error',patience=2, verbose=1, factor=0.5, min_lr=0.000000001)
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=30,restore_best_weights=True,verbose=0)


In [ ]:
weights_file = 'Tsne_mnist_mlp.h5'

if os.path.exists(weights_file):
    Autoencoder.load_weights(weights_file)
    my_history=np.load('./Models/my_history.npy',allow_pickle='TRUE').item()
    print('Loaded weights!')
else:
    history_keras =  Autoencoder.fit(x_train,x_train, epochs=200, validation_data=(x_val,x_val), callbacks=[early_stopping],verbose=1)
    Autoencoder.save_weights('space_mlp.h5')
    np.save('my_history.npy',history_keras.history)
    my_history=history_keras.history